# Frontend State & Data Management

Comprehensive guide to managing state and data in modern frontend applications.

## Topics Covered
| Section | Description |
|---------|-------------|
| Local vs Global State | When to use component vs application state |
| State Management Patterns | Flux, Redux, Zustand architectures |
| React Context | Built-in state sharing mechanism |
| Data Fetching | REST, GraphQL, React Query strategies |
| Caching & Optimistic Updates | Performance optimization techniques |
| Form State Management | Handling complex form interactions |

## 1. Local vs Global State

### State Categories

```
┌─────────────────────────────────────────────────────────────────────┐
│                        STATE TAXONOMY                               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  LOCAL STATE                    GLOBAL STATE                        │
│  ───────────                    ────────────                        │
│  • UI state (open/closed)       • User authentication               │
│  • Form input values            • Shopping cart                     │
│  • Toggle/accordion state       • Theme/locale preferences          │
│  • Hover/focus states           • Cached API responses              │
│                                                                     │
│  ┌─────────┐                    ┌──────────────────────┐            │
│  │Component│ ←── owns           │     Global Store     │            │
│  │  State  │                    │  ┌────┬────┬────┐    │            │
│  └─────────┘                    │  │Auth│Cart│Theme│   │            │
│       │                         │  └────┴────┴────┘    │            │
│       ▼                         └──────────┬───────────┘            │
│  useState()                                │                        │
│  useReducer()                    ┌─────────┼─────────┐              │
│                                  ▼         ▼         ▼              │
│                              [Comp A] [Comp B] [Comp C]             │
└─────────────────────────────────────────────────────────────────────┘
```

### Decision Matrix

| Factor | Use Local State | Use Global State |
|--------|-----------------|------------------|
| Scope | Single component | Multiple components |
| Lifetime | Component mount/unmount | App lifecycle |
| Persistence | Not needed | Often required |
| Sharing | No sibling access | Cross-tree access |

## 2. State Management Patterns

### Flux Architecture (Foundation)

```
┌────────────────────────────────────────────────────────────────┐
│                    FLUX UNIDIRECTIONAL FLOW                    │
├────────────────────────────────────────────────────────────────┤
│                                                                │
│     ┌────────┐      ┌────────────┐      ┌───────┐             │
│     │ Action │─────▶│ Dispatcher │─────▶│ Store │             │
│     └────────┘      └────────────┘      └───┬───┘             │
│          ▲                                  │                  │
│          │                                  ▼                  │
│          │                            ┌──────────┐            │
│          └────────────────────────────│   View   │            │
│                  (user interaction)   └──────────┘            │
│                                                                │
└────────────────────────────────────────────────────────────────┘
```

### Redux Pattern

```
┌─────────────────────────────────────────────────────────────────┐
│                        REDUX DATA FLOW                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   UI ──dispatch──▶ Action ──▶ Middleware ──▶ Reducer ──▶ Store │
│    ▲                              │              │         │    │
│    │                              │              ▼         │    │
│    │                          (async)     Pure Function    │    │
│    │                           thunk      (state, action)  │    │
│    │                           saga         => newState    │    │
│    │                                                       │    │
│    └───────────────────subscribe/selector──────────────────┘    │
│                                                                 │
│   // Reducer Example                                            │
│   const todoReducer = (state = [], action) => {                 │
│     switch (action.type) {                                      │
│       case 'ADD_TODO':                                          │
│         return [...state, action.payload];                      │
│       case 'REMOVE_TODO':                                       │
│         return state.filter(t => t.id !== action.payload);      │
│       default:                                                  │
│         return state;                                           │
│     }                                                           │
│   };                                                            │
└─────────────────────────────────────────────────────────────────┘
```

### Zustand (Modern Lightweight)

```javascript
// Simple store with Zustand
import { create } from 'zustand';

const useStore = create((set, get) => ({
  // State
  count: 0,
  users: [],
  
  // Actions (mutate directly - Immer under the hood)
  increment: () => set(state => ({ count: state.count + 1 })),
  
  // Async actions
  fetchUsers: async () => {
    const res = await fetch('/api/users');
    set({ users: await res.json() });
  },
  
  // Computed values via selectors
  getUserById: (id) => get().users.find(u => u.id === id)
}));

// Usage in component
const count = useStore(state => state.count);  // Auto re-render on change
```

### Comparison

| Feature | Redux | Zustand | Jotai | Recoil |
|---------|-------|---------|-------|--------|
| Boilerplate | High | Low | Low | Medium |
| Bundle Size | ~2KB | ~1KB | ~2KB | ~20KB |
| DevTools | Excellent | Good | Good | Good |
| Learning Curve | Steep | Easy | Easy | Medium |
| Best For | Enterprise | Small-Med | Atomic state | Complex graphs |

## 3. React Context API

```
┌───────────────────────────────────────────────────────────────────┐
│                   CONTEXT PROVIDER PATTERN                        │
├───────────────────────────────────────────────────────────────────┤
│                                                                   │
│   ┌─────────────────────────────────────────────────────────┐    │
│   │              <ThemeContext.Provider>                     │    │
│   │  ┌───────────────────────────────────────────────────┐  │    │
│   │  │              <AuthContext.Provider>                │  │    │
│   │  │  ┌─────────────────────────────────────────────┐  │  │    │
│   │  │  │                   <App />                    │  │  │    │
│   │  │  │      ┌────────────────────────────┐         │  │  │    │
│   │  │  │      │ useContext(ThemeContext) ✓ │         │  │  │    │
│   │  │  │      │ useContext(AuthContext)  ✓ │         │  │  │    │
│   │  │  │      └────────────────────────────┘         │  │  │    │
│   │  │  └─────────────────────────────────────────────┘  │  │    │
│   │  └───────────────────────────────────────────────────┘  │    │
│   └─────────────────────────────────────────────────────────┘    │
│                                                                   │
└───────────────────────────────────────────────────────────────────┘
```

```javascript
// Context with useReducer pattern
const CartContext = createContext();

function cartReducer(state, action) {
  switch (action.type) {
    case 'ADD_ITEM':
      return { ...state, items: [...state.items, action.payload] };
    case 'CLEAR':
      return { ...state, items: [] };
    default:
      return state;
  }
}

export function CartProvider({ children }) {
  const [state, dispatch] = useReducer(cartReducer, { items: [] });
  
  // Memoize to prevent unnecessary re-renders
  const value = useMemo(() => ({ state, dispatch }), [state]);
  
  return (
    <CartContext.Provider value={value}>
      {children}
    </CartContext.Provider>
  );
}

// Custom hook for clean API
export const useCart = () => useContext(CartContext);
```

### ⚠️ Context Pitfalls

| Issue | Problem | Solution |
|-------|---------|----------|
| Re-render cascade | All consumers re-render on any change | Split contexts by update frequency |
| Prop drilling avoided but... | Over-using context for all state | Use context for truly global state only |
| No selector support | Can't subscribe to partial state | Use libraries like `use-context-selector` |

## 4. Data Fetching Strategies

### REST vs GraphQL

```
┌────────────────────────────────────────────────────────────────────┐
│                    REST vs GRAPHQL                                 │
├─────────────────────────┬──────────────────────────────────────────┤
│          REST           │              GraphQL                     │
├─────────────────────────┼──────────────────────────────────────────┤
│                         │                                          │
│  GET /users/123         │  query {                                 │
│  GET /users/123/posts   │    user(id: 123) {                       │
│  GET /users/123/friends │      name                                │
│                         │      posts { title }     ◄── 1 request   │
│  ▲ 3 requests           │      friends { name }                    │
│  ▲ Over-fetching        │    }                                     │
│                         │  }                                       │
│                         │                                          │
│  Response: Full object  │  Response: Only requested fields         │
│  (includes unused data) │  (exact shape you need)                  │
│                         │                                          │
└─────────────────────────┴──────────────────────────────────────────┘
```

### React Query / TanStack Query

```javascript
// Declarative data fetching with automatic caching
import { useQuery, useMutation, useQueryClient } from '@tanstack/react-query';

function UserProfile({ userId }) {
  // Automatic caching, deduplication, background refetch
  const { data, isLoading, error } = useQuery({
    queryKey: ['user', userId],
    queryFn: () => fetch(`/api/users/${userId}`).then(r => r.json()),
    staleTime: 5 * 60 * 1000,    // Fresh for 5 minutes
    cacheTime: 30 * 60 * 1000,   // Keep in cache 30 minutes
  });
  
  if (isLoading) return <Skeleton />;
  if (error) return <ErrorBoundary error={error} />;
  return <Profile user={data} />;
}
```

### Fetching Strategy Comparison

| Strategy | Caching | Dedup | Retry | SSR | Best For |
|----------|---------|-------|-------|-----|----------|
| fetch + useEffect | Manual | No | Manual | No | Simple cases |
| SWR | ✓ | ✓ | ✓ | ✓ | Vercel/Next.js |
| React Query | ✓ | ✓ | ✓ | ✓ | Complex apps |
| Apollo Client | ✓ | ✓ | ✓ | ✓ | GraphQL apps |
| RTK Query | ✓ | ✓ | ✓ | ✓ | Redux ecosystems |

## 5. Caching & Optimistic Updates

### Caching Architecture

```
┌─────────────────────────────────────────────────────────────────────┐
│                    MULTI-LAYER CACHING                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Request ───▶ ┌───────────────┐                                   │
│                │  Memory Cache  │◄── Query cache (React Query)     │
│                │   (fastest)    │    Normalized cache (Apollo)     │
│                └───────┬────────┘                                   │
│                   miss │                                            │
│                        ▼                                            │
│                ┌───────────────┐                                   │
│                │ Browser Cache  │◄── localStorage, IndexedDB       │
│                │  (persistent)  │    Service Worker cache          │
│                └───────┬────────┘                                   │
│                   miss │                                            │
│                        ▼                                            │
│                ┌───────────────┐                                   │
│                │   CDN/Edge    │◄── Static assets, API responses   │
│                └───────┬────────┘                                   │
│                   miss │                                            │
│                        ▼                                            │
│                ┌───────────────┐                                   │
│                │ Origin Server │                                   │
│                └───────────────┘                                   │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### Optimistic Updates Pattern

```javascript
// Optimistic update with rollback
const queryClient = useQueryClient();

const mutation = useMutation({
  mutationFn: updateTodo,
  
  // 1. Optimistically update before server response
  onMutate: async (newTodo) => {
    await queryClient.cancelQueries(['todos']);
    const previous = queryClient.getQueryData(['todos']);
    
    queryClient.setQueryData(['todos'], old => 
      old.map(t => t.id === newTodo.id ? newTodo : t)
    );
    
    return { previous };  // Context for rollback
  },
  
  // 2. Rollback on error
  onError: (err, newTodo, context) => {
    queryClient.setQueryData(['todos'], context.previous);
    toast.error('Update failed, reverting...');
  },
  
  // 3. Refetch to ensure consistency
  onSettled: () => queryClient.invalidateQueries(['todos'])
});
```

### Cache Invalidation Strategies

| Strategy | Trigger | Use Case |
|----------|---------|----------|
| Time-based (TTL) | `staleTime` expiry | Static data |
| Event-based | After mutation | CRUD operations |
| Manual | User action (pull-to-refresh) | User-controlled refresh |
| Websocket | Real-time server push | Collaborative apps |

## 6. Form State Management

### Form State Complexity

```
┌─────────────────────────────────────────────────────────────────────┐
│                      FORM STATE LAYERS                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐              │
│   │   VALUES    │   │   ERRORS    │   │   TOUCHED   │              │
│   ├─────────────┤   ├─────────────┤   ├─────────────┤              │
│   │ email: ''   │   │ email: null │   │ email: false│              │
│   │ password: ''│   │ password:   │   │ password:   │              │
│   │             │   │   'Required'│   │   true      │              │
│   └─────────────┘   └─────────────┘   └─────────────┘              │
│                                                                     │
│   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐              │
│   │   DIRTY     │   │ SUBMITTING  │   │  VALIDITY   │              │
│   ├─────────────┤   ├─────────────┤   ├─────────────┤              │
│   │ form: true  │   │ isSubmitting│   │ isValid:    │              │
│   │ fields: {}  │   │   : false   │   │   false     │              │
│   └─────────────┘   └─────────────┘   └─────────────┘              │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### React Hook Form (Recommended)

```javascript
import { useForm } from 'react-hook-form';
import { zodResolver } from '@hookform/resolvers/zod';
import { z } from 'zod';

// Schema validation
const schema = z.object({
  email: z.string().email('Invalid email'),
  password: z.string().min(8, 'Minimum 8 characters'),
});

function LoginForm() {
  const { register, handleSubmit, formState: { errors, isSubmitting } } = useForm({
    resolver: zodResolver(schema),
    mode: 'onBlur',  // Validate on blur
  });
  
  const onSubmit = async (data) => {
    await loginAPI(data);
  };
  
  return (
    <form onSubmit={handleSubmit(onSubmit)}>
      <input {...register('email')} />
      {errors.email && <span>{errors.email.message}</span>}
      
      <input type="password" {...register('password')} />
      {errors.password && <span>{errors.password.message}</span>}
      
      <button disabled={isSubmitting}>Login</button>
    </form>
  );
}
```

### Form Library Comparison

| Library | Re-renders | Bundle | Validation | Best For |
|---------|------------|--------|------------|----------|
| React Hook Form | Minimal | ~9KB | Zod, Yup | Performance-critical |
| Formik | On every change | ~15KB | Yup | Full-featured |
| React Final Form | Configurable | ~8KB | Custom | Fine control |
| Native (useState) | On every change | 0KB | Manual | Simple forms |

## 7. Architecture Decision Guide

### State Management Selection Flowchart

```
┌─────────────────────────────────────────────────────────────────────┐
│              CHOOSING THE RIGHT STATE SOLUTION                      │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│                     Is it UI state?                                 │
│                          │                                          │
│               ┌──────────┴──────────┐                               │
│               ▼                     ▼                               │
│             [YES]                 [NO]                              │
│               │                     │                               │
│     Is it shared across            Is it server data?               │
│     distant components?                 │                           │
│          │                    ┌─────────┴─────────┐                 │
│     ┌────┴────┐               ▼                   ▼                 │
│     ▼         ▼             [YES]               [NO]                │
│   [NO]      [YES]             │                   │                 │
│     │         │         React Query/         URL State              │
│ useState   Context      SWR/Apollo        (query params)            │
│            or Zustand                                               │
│                                                                     │
│   ──────────────────────────────────────────────────────────────   │
│                                                                     │
│   RECOMMENDED STACK:                                                │
│   • UI State: useState/useReducer (local), Zustand (global)         │
│   • Server State: React Query / TanStack Query                      │
│   • URL State: React Router / Next.js router                        │
│   • Form State: React Hook Form + Zod                               │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### Key Principles

1. **Colocation**: Keep state as close to where it's used as possible
2. **Single Source of Truth**: Server data belongs in query cache, not Redux
3. **Derived State**: Calculate values instead of storing them
4. **Minimal State**: Store only what can't be derived
5. **Immutability**: Never mutate state directly